In [1]:
import random

In [2]:
def is_feasible(solution):
    return any(solution)

In [3]:
class Individual:
    def __init__(self, cost, facility_cost):
        num_resources = len(facility_cost)
        self.code = [random.random() < 0.5 for _ in range(num_resources)]
        self.fitness = self.calc_fitness(facility_cost, cost)
        
    def calc_fitness(self, facility_cost, cost):
        if not is_feasible(self.code):
            return float('-inf') # moze suma svih cost i facility_cost

        value = 0
        for i, s in enumerate(self.code):
            if s:
                value += facility_cost[i]

        for i in range(len(cost)):
            min_cost = float('inf')
            for j in range(len(cost[i])):
                if self.code[j] and cost[i][j] < min_cost:
                    min_cost = cost[i][j]
            value += min_cost

        return -value

In [4]:
def selection(population, tournament_size):
#     TODO roulette and rang roulette
    chosen = random.sample(population, tournament_size)
    return max(chosen, key=lambda x: x.fitness)

In [5]:
def crossover(parent1, parent2, child1, child2):
    random_pos = random.randrange(0, len(parent1.code))
    
    child1.code[:random_pos] = parent1.code[:random_pos]
    child1.code[random_pos:] = parent2.code[random_pos:]
    
    child2.code[:random_pos] = parent2.code[:random_pos]
    child2.code[random_pos:] = parent1.code[random_pos:]

In [6]:
def mutation(individual, mutation_prob):
    for i in range(len(individual.code)):
        if random.random() < mutation_prob:
            individual.code[i] = not individual.code[i]

In [19]:
def ga(cost, facility_cost, population_size, num_generations, tournament_size, elitism_size, mutation_prob):
    population = [Individual(cost, facility_cost) for _ in range(population_size)]
    new_population = population.copy()
    
#     dva nacina za neponavljanje roditelja - za domaci dodati u selection
#     xs = [1,2,3,4,5]
#     p1 = random.choice(xs)
# #     p2 = random.choice(xs.difference({p1}))
#     while True:
#         p2 = random.choice(xs)
#         if p1 != p2:
#             break
    
    for i in range(num_generations):
        population.sort(key=lambda x: x.fitness, reverse=True)
        new_population[:elitism_size] = population[:elitism_size]
        for j in range(elitism_size, population_size, 2):
            parent1 = selection(population, tournament_size)
            parent2 = selection(population, tournament_size) # TODO razlikuju se
            
            crossover(parent1, parent2, child1=new_population[j], child2=new_population[j+1])

            mutation(new_population[j], mutation_prob)
            mutation(new_population[j+1], mutation_prob)
            
            new_population[j].fitness = new_population[j].calc_fitness(facility_cost, cost)
            new_population[j+1].fitness = new_population[j+1].calc_fitness(facility_cost, cost)
        
        population = new_population.copy()
    return max(population, key=lambda x: x.fitness)

In [20]:
def read_line_of_ints(f):
    return [int(x) for x in f.readline().split()]

In [21]:
def read_instance(file_path: str):
    with open(file_path, 'r') as f:
        m, n = read_line_of_ints(f)
        cost = [read_line_of_ints(f) for _ in range(m)]
        facility_cost = read_line_of_ints(f)
        return cost, facility_cost

In [22]:
cost, facility_cost = read_instance('uflp1.txt')

In [23]:
best_individual = ga(
    cost=cost,
    facility_cost=facility_cost,
    population_size=100,
    num_generations=10,
    tournament_size=7,
    elitism_size=10,
    mutation_prob=0.05,
  )

In [24]:
best_individual.code

[True, False, False]

In [25]:
best_individual.fitness

-34

In [26]:
import numpy as np

In [27]:
def read_bk_instance(file_path: str = 'BildeKrarup/B/B1.1'):
    with open(file_path, 'r') as f:
        f.readline()
        num_resources, num_users, _ = read_line_of_ints(f)
        cost = [[None for _ in range(num_users)] for _ in range(num_resources)]
        facility_cost = [None for _ in range(num_resources)]
        for i in range(num_resources):
            ints = read_line_of_ints(f)
            cost[i] = ints[2:]
            facility_cost[i] = ints[1]
    return np.array(cost).transpose(), facility_cost

In [28]:
cost, facility_cost = read_bk_instance()

In [32]:
best_individual = ga(
    cost=cost,
    facility_cost=facility_cost,
    population_size=100,
    num_generations=100,
    tournament_size=7,
    elitism_size=10,
    mutation_prob=0.05,
  )

In [33]:
best_individual.fitness

-24922